In [1]:
"""
RQS_eval.py

Reads a specified JSON file and iterates through each object's 'turns' array to find a turn where 'isuser' is true and its following turn.

Usage:
    python RQS_eval.py outputs/llm_responses.json outputs/llm_responses_rqs.json

Arguments:
    --file_path: Path to the JSON file.
    --output_path: Path where the modified JSON file will be saved.
"""

from tools.api_request import request_gpt as request_llm
from tools.db_detail import db_getdesc
from tools.sql_execute import sqlite_execute as execute
import json
from tqdm import tqdm

def sql_evoke(query,db_name):
    result, execution_time ,executable = execute("datasets/cosql_dataset/database/"+db_name+"/"+db_name+".sqlite",query)
    return result 

def get_example(db_name):
    sql_query = "SELECT name FROM sqlite_master WHERE type='table';"
    result = sql_evoke(sql_query,db_name)
    column_example=""
    for table_name in result:
        column_example = column_example + table_name[0] + ":\n"
        sql_get_eg = "SELECT * FROM "+ table_name[0] +" LIMIT 3;"
        table_eg = sql_evoke(sql_get_eg,db_name)
        for table_data in table_eg:
            column_example = column_example + '('
            for column_data in table_data: 
                column_example = column_example + str(column_data) +','
            column_example = column_example[:-1] + ')\n'
    return column_example
    

def ask_ai(db_name, question, answer_pred, answer_gold):
    description = db_getdesc(db_name)
    column_example = get_example(db_name)
    template = """
{database_description}

{user_question}

{system_response}

{reference_answer}

Evaluate the quality of the system's response based on the following criteria. Assign 2 points directly if a criterion does not apply.
Relevance:
0 points: The response is completely irrelevant.
1 point: The response is partially relevant but misses key details.
2 points: The response is fully relevant and addresses the question adequately.
Clarity:
0 points: The response is incomprehensible.
1 point: The response is mostly clear with minor ambiguities.
2 points: The response is very clear and easy to understand.
Completeness:
0 points: The response does not address the question at all.
1 point: The response covers most aspects of the question but lacks some details.
2 points: The response thoroughly addresses all aspects of the question.
Accuracy:
0 points: The response contains factually incorrect information.
1 point: The response is partially accurate with some errors.
2 points: The response is completely accurate.
Utility:
0 points: The response does not meet the user's needs or explain the context of the question.
1 point: The response somewhat meets the user's needs and provides partial explanations.
2 points: The response excellently meets the user's needs and clearly explains the context or ambiguity of the question.
Task:
Classify the Response: Determine if the system response is 'improper'(Non-SQL based user questions), 'unanswerable'(unachievable under existing conditions), or 'ambiguous'(Lack of clarity).
Evaluate Each Criterion: Provide a detailed rationale for the score assigned to each criterion.
Calculate the Total Score: Sum the scores for all criteria.(10 points for a direct greeting alone)

Output Format:
{{
  "AnswerType": "",(text only)
  "Rationale": "",(text only)
  "Score": ""(An integer from 0 to 10)
}}
    """
    filled_template = template.format(
        database_description="Database Description:"+ "\nDatabase schema:\n" + description + "\nExamples for each table:"+ column_example,
        user_question="User Question:" + question,
        system_response="System Response:" + answer_pred,
        reference_answer="Reference Answer:" + answer_gold
    )
    
    # print(filled_template)

    messages = [{"role": "user", "content": filled_template}]
    max_attempts = 5
    attempt = 0
    while attempt < max_attempts:
        llm_response = request_llm(messages)
        print("LLM Response:", llm_response)
        select_pos = llm_response.find('{')
        colon_pos = llm_response.find('}', select_pos)
        if select_pos != -1 and colon_pos != -1:
            llm_response = llm_response[select_pos:colon_pos+1].replace('\n',' ')
            print("formatted json "+llm_response)
        try:
            response_data = json.loads(llm_response)
            type_ai = response_data.get("AnswerType", "")
            rqs_ai = response_data.get("Score", 0)
            rationale_ai = response_data.get("Rationale", "")
            
            # 检查返回的类型和分数是否符合预期
            if type_ai in ["improper", "unanswerable", "ambiguous"] and int(rqs_ai) >= 0 and int(rqs_ai) <= 10:
                return type_ai, rqs_ai, rationale_ai
            else:
                raise ValueError("Response type or score out of expected range.")
        except (json.JSONDecodeError, KeyError, ValueError, TypeError, Exception) as e:
            print("\033[91mRQS_eval.py::: Retry Reason: {}\033[0m".format(str(e)))  # 红色字体提示重试原因
            attempt += 1
    return "error", 0, "error"

def process_turns(file_path, output_path):

    # Open and read the JSON file
    with open(file_path, 'r') as file:
        data = json.load(file)
    
    # Iterate over each object in the file
    for entry in tqdm(data):
        # Check if the 'turns' key exists
        print("__________________")
        if 'turns' in entry:
            turns = entry['turns']
            db_name = entry['db_name']
            length = len(turns)
            # Iterate over each turn in 'turns'
            for i in range(length):
                # Check if the current turn is a user turn
                if turns[i].get('isuser', False):
                    print(i//2)
                    # Output the current user's turn
                    print(turns[i].get('type', ''), "--", turns[i].get('text', ''))
                    # Check and process the next turn (if it exists)
                    if i + 1 < length:
                        next_turn = turns[i + 1]
                        predict_text = next_turn.get('predict', '')
                        # Find the positions of SELECT and the semicolon
                        select_pos = predict_text.upper().find('SELECT')
                        colon_pos = predict_text.find(';', select_pos)
                        if select_pos != -1 and colon_pos != -1:
                            predict_sql = predict_text[select_pos:colon_pos].replace('\n','')
                        elif select_pos != -1:
                            predict_sql = predict_text[select_pos:].replace('\n',' ')
                        else:
                            predict_sql = ""
                        # Store the result in a new field 'predict_sql'
                        next_turn['predict_sql'] = predict_sql
                        # Calculate the ratio of the extracted SQL to the entire predict field
                        if len(predict_text) == 0:
                            ratio = 0
                        else:
                            ratio = len(predict_sql) / len(predict_text)
                            
                        if predict_sql != "" and ratio >= 0.5:
                                next_turn['predict_type'] = 'answerable'
                                if turns[i].get('type', '') == 'answerable':
                                    next_turn['RQS'] = "N/A"
                                else:
                                    next_turn['RQS'] = 0
                        else:
                            next_turn['predict_type'] = 'not answerable'
                            # Ask LLM, Get categorized and RQS scored based on database, questions, answers, gold answer
                            type_ai, rqs_ai, rationale_ai = ask_ai(db_name,turns[i].get('text', ''),predict_text,next_turn.get('text', '')) 
                            next_turn['predict_type'] = type_ai
                            next_turn['RQS'] = rqs_ai
                            next_turn['RQS_Rationale'] = rationale_ai
                        print("Next Turn predict_sql:", predict_sql)
                        print("Predict Type:", next_turn['predict_type'])
                    else:
                        print("Next Turn does not exist.")
    
    # Save the modified data to a new JSON file
    with open(output_path, 'w') as outfile:
        json.dump(data, outfile, indent=4)

# Example usage
input_file_path = 'outputs/gemini-1-Copy1.5-flash-llm.json'
output_file_path = 'outputs/gpt4打分的gemini输出.json'
process_turns(input_file_path, output_file_path)


  0%|                                                                                                                                                                                                                                                                             | 0/148 [00:00<?, ?it/s]

__________________
0
answerable -- How many students in the "Student" table are over the age of 20?
Next Turn predict_sql: SELECT COUNT(*)FROM StudentWHERE Age > 20
Predict Type: answerable
1
answerable -- How many of those students own pets?
Next Turn predict_sql: SELECT COUNT(DISTINCT S.StuID)FROM Student AS SJOIN Has_Pet AS HP ON S.StuID = HP.StuIDWHERE S.Age > 20
Predict Type: answerable
__________________
0
answerable -- Can you list all the continents?
Next Turn predict_sql: SELECT Continent FROM continents
Predict Type: answerable
1
answerable -- Okay, now can you list the number of countries belonging to each continent, including the continent id as well?
Next Turn predict_sql: SELECT c.ContId, c.Continent, COUNT(DISTINCT co.CountryId) AS NumberOfCountriesFROM continents cJOIN countries co ON c.ContId = co.ContinentGROUP BY c.ContId, c.ContinentORDER BY c.ContId
Predict Type: answerable
2
improper -- Thanks!
request gpt-4-turbo


  1%|███▌                                                                                                                                                                                                                                                                 | 2/148 [00:05<06:09,  2.53s/it]

LLM Response: {
  "AnswerType": "improper",
  "Rationale": "The user's input 'Thanks!' is a simple expression of gratitude, not a question or request for information. The system's response 'You're welcome! Let me know if you have any other questions.' is a polite and appropriate closure to the conversation, matching the user's tone and context. It does not need to address any specific content from the database as no question was asked about it.",
  "Score": "10"
}
formatted json {   "AnswerType": "improper",   "Rationale": "The user's input 'Thanks!' is a simple expression of gratitude, not a question or request for information. The system's response 'You're welcome! Let me know if you have any other questions.' is a polite and appropriate closure to the conversation, matching the user's tone and context. It does not need to address any specific content from the database as no question was asked about it.",   "Score": "10" }
Next Turn predict_sql: 
Predict Type: improper
______________

  2%|█████▎                                                                                                                                                                                                                                                               | 3/148 [00:09<07:38,  3.16s/it]

...

LLM Response: {
  "AnswerType": "improper",
  "Rationale": "The user's input 'Thanks!' is a polite closure to a conversation rather than a query requiring a detailed response or further information. The system's response 'You're welcome! Is there anything else I can help you with? 😊' is appropriate and polite, effectively closing the interaction while leaving room for any additional queries the user might have. This type of exchange does not involve the database schema or require SQL-based answers, hence it is classified as 'improper' for the context of SQL-based queries but is proper in a general conversational context.",
  "Score": "10"
}
formatted json {   "AnswerType": "improper",   "Rationale": "The user's input 'Thanks!' is a polite closure to a conversation rather than a query requiring a detailed response or further information. The system's response 'You're welcome! Is there anything else I can help you with? 😊' is appropriate and polite, effectively closing the interaction wh

 99%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎ | 147/148 [18:24<00:09,  9.54s/it]

LLM Response: {
  "AnswerType": "improper",
  "Rationale": "The user's input 'Thanks!' is a simple expression of gratitude and does not pose a question or require further information related to the database schema provided. The system's response 'You're welcome! Is there anything else I can help you with?' is appropriate as it acknowledges the user's thanks and opens the floor for any further queries the user might have. This interaction is typical in conversational exchanges and does not pertain to the database content or require SQL-based responses. Therefore, the response is classified as 'improper' because it does not relate to SQL queries or database information but is still a correct conversational reply.",
  "Score": "10"
}
formatted json {   "AnswerType": "improper",   "Rationale": "The user's input 'Thanks!' is a simple expression of gratitude and does not pose a question or require further information related to the database schema provided. The system's response 'You're welc

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 148/148 [18:41<00:00,  7.58s/it]

LLM Response: {
  "AnswerType": "improper",
  "Rationale": "The user's input 'No.' is not a question or a statement that requires further information or action related to the database schema provided. It seems to be a response to a non-visible prompt or question. The system's response 'Okay, let me know if you have any other questions.' is appropriate as it acknowledges the user's input and remains open for further interaction, which is suitable for a conversational interface. However, it does not relate to the database content or require SQL-based interaction, making the response type 'improper' for the context of database querying but proper in a general conversational context.",
  "Score": "10"
}
formatted json {   "AnswerType": "improper",   "Rationale": "The user's input 'No.' is not a question or a statement that requires further information or action related to the database schema provided. It seems to be a response to a non-visible prompt or question. The system's response 'Oka